In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
df_train = pd.read_parquet("./data/yellow_tripdata_2022-01.parquet")
df_val = pd.read_parquet("./data/yellow_tripdata_2022-02.parquet")

In [19]:
df_train["duration"] = pd.to_datetime(df_train.tpep_dropoff_datetime) - pd.to_datetime(df_train.tpep_pickup_datetime)
df_train.duration = df_train.duration.apply(lambda d: d.total_seconds() / 60)

df_without_outliers = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

In [23]:
df_val["duration"] = pd.to_datetime(df_val.tpep_dropoff_datetime) - pd.to_datetime(df_val.tpep_pickup_datetime)
df_val.duration = df_val.duration.apply(lambda d: d.total_seconds() / 60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [24]:
df_without_outliers.duration.count() / df_train.duration.count()

0.9827547930522406

In [25]:
categorical = ["PULocationID","DOLocationID"]

In [9]:
df_without_outliers[categorical] = df_without_outliers[categorical].astype(str)
train_dicts = df_without_outliers[categorical].to_dict(orient="records")

In [26]:
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient="records")

In [27]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [28]:
target = "duration"
Y_train = df_without_outliers[target].values
Y_val = df_val[target].values

In [29]:
lr = LinearRegression()
lr.fit(X_train,Y_train)

LinearRegression()

In [30]:
Y_pred = lr.predict(X_val)

In [31]:
mean_squared_error(Y_val, Y_pred, squared=False)

7.786409085078911